<a href="https://colab.research.google.com/github/RealGIS-creator/scripts_realgis_banistmo_2025/blob/main/Preparacion_InsertFolios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os

all_files = os.listdir()

csv_files = [f for f in all_files if f.endswith('.csv')]

df_list = []

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file)
        df_list.append(df)
        print(f"Successfully read {csv_file}")
    except Exception as e:
        print(f"Error reading {csv_file}: {e}")

if df_list:
    combined_df = pd.concat(df_list, ignore_index=True)

    print("\nCombined DataFrame (first 5 rows):")
    print(combined_df.head())

else:
    print("No CSV files found to combine.")


combined_df.to_csv('DatosFolios_27052025.csv', index=False)

from google.colab import files
files.download('DatosFolios_27052025.csv')

Successfully read DatosFolio3.csv
Successfully read DatosFolio2.csv
Successfully read DatosFolio1.csv
Successfully read DatosFolio5.csv
Successfully read DatosFolio4.csv

Combined DataFrame (first 5 rows):
  IDENTIFICACION                              FOLIO / FINCA / FICHA   \
0      2-724-2339  (INMUEBLE) PANAMÁ Código de Ubicación 8716  Fo...   
1       2-730-756  (INMUEBLE) PENONOMÉ Código de Ubicación 2501  ...   
2       9-710-479  (INMUEBLE) SANTIAGO Código de Ubicación 9908  ...   
3       8-901-296  (INMUEBLE) ARRAIJÁN Código de Ubicación 8003  ...   
4      8-817-2007  (INMUEBLE) LA CHORRERA Código de Ubicación 861...   

                                    PROPIETARIO   \
0       LUIS ANTONIO GOMEZ SAMANIEGO (Propiedad)   
1             ANA GABRIELA ARCIA . . (Propiedad)   
2        RUBEN DARIO CASTILLO IBARRA (Propiedad)   
3  jose ricauther martinez caballero (Propiedad)   
4      YERALDINE ELIZABETH RODRIGUEZ (Propiedad)   

                                          DOMICI

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install unidecode
import pandas as pd
import re
import unidecode

try:
    df = pd.read_csv('/content/DatosFolios_27052025.csv', encoding='utf-8', on_bad_lines='warn')
except UnicodeDecodeError:
    try:
        df = pd.read_csv('DatosFolios_27052025.csv', encoding='utf-8', on_bad_lines='warn')
    except UnicodeDecodeError:
        df = pd.read_csv('DatosFolios_27052025.csv', encoding='ISO-8859-1', on_bad_lines='warn')

for col in df.columns:
    new_col = ''.join(filter(str.isalnum, col))
    df = df.rename(columns={col: new_col})
    if df[new_col].dtype == 'object':
        df[new_col] = df[new_col].astype(str).apply(lambda x: unidecode.unidecode(x) if isinstance(x, str) else x).str.replace('[^a-zA-Z0-9\s]', '', regex=True).str.upper().str.replace('NAN','',regex=True)


if 'FOLIOFINCAFICHA' in df.columns:
    df['FINCA_Part1'] = df['FOLIOFINCAFICHA'].str.extract('(\d+)', expand=False)
    df['FINCA_Part2'] = df['FOLIOFINCAFICHA'].str.extract('(?:FOLIO REAL NO\s*)(\d+)', expand=False)
    df['FINCA_Part3'] = df['FOLIOFINCAFICHA'].str.extract('(?:FOLIO REAL NO\s*\d+\s*)(\d+)', expand=False)

df.to_csv('/content/DatosFoliosLimpios.csv', index=False)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 12.3 MB/s eta 0:00:00


In [ ]:
acreditados_df = pd.read_csv('ACREDITADOS.csv')
folios_df = pd.read_csv('DatosFoliosLimpios.csv')

acreditados_df['acreditadoidenti'] = acreditados_df['acreditadoidenti'].astype(str).str.replace('-', '').str.strip().str.replace('CO', '').str.replace('C0', '')

merged_df = pd.merge(acreditados_df, folios_df, left_on='acreditadoidenti', right_on='IDENTIFICACION', how='inner')

print(merged_df.head())

merged_df = merged_df[~merged_df.duplicated(subset=['FINCA_Part2'], keep=False)]

merged_df.to_csv('/content/merge_datosFolioslimpios2_26052025.csv', index= False)

   acreditado_id  acreditadonum acreditadoidenti               acreditadonom  \
0          19404         343981          E887537                  YIJUAN GAN   
1          19404         343981          E887537                  YIJUAN GAN   
2          19404         343981          E887537                  YIJUAN GAN   
3          19414       55002760          E860857        SOU LAN CHOW DE CHEN   
4          19452       98865977          2861041  OMAR ANTONIO GRAELL RANGEL   

   tipopersona_id  acreditadonumcuen              acrefechcre acreusucre  \
0               4   5521621502871550  2025-04-04 18:19:01.302      admin   
1               4   5521621502871550  2025-04-04 18:19:01.302      admin   
2               4   5521621502871550  2025-04-04 18:19:01.302      admin   
3               4   5521621502888560  2025-04-04 18:19:01.302      admin   
4               1   4343300000366540  2025-04-04 18:19:01.302      admin   

  acreipcre              acrefechmod  ... acreobjcre acreobjmo